In [37]:
import pandas as pd
import numpy as np
import random

from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter

### Create a small example dataset

In [38]:
data = [5, 21, 1, 29, 32, 37, 10, 20, 10, 26, 2, 37, 34, 11, 22, 36, 12, 20, 31, 25]

In [39]:
df = pd.DataFrame(data, columns=['Number'])

### Creating a small validation set

In [40]:
validation = [22, 11, 7, 2, 32]

In [41]:
df_val = pd.DataFrame(validation, columns=['Number'])

Ground truth array for the validation set. 

22 -> not prime [0]<br>
11 -> prime [1]<br>
7 -> prime [1]<br>
2 -> prime [1]<br>
32 -> not prime [0]<br>

In [42]:
true_labels = np.array([0, 1, 1, 1, 0])

In [43]:
df_tl = pd.DataFrame(true_labels, columns=['Number'])

### Labeling functions

In [44]:
ABSTAIN = -1
NON_PRIME = 0
PRIME = 1

In [45]:
@labeling_function()
def is_odd(record):
    if record["Number"]%2 == 1:
        return ABSTAIN
    else:
        return NON_PRIME

In [46]:
@labeling_function()
def is_even(record):
    if record["Number"]%2 == 0:
        return NON_PRIME
    else:
        return ABSTAIN

In [47]:
@labeling_function()
def is_two(record):
    if record["Number"] == 2:
        return PRIME
    else:
        return ABSTAIN

In [48]:
@labeling_function()
def is_known_prime(record):
    known_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
    if record["Number"] in known_primes:
        return PRIME
    else:
        return ABSTAIN

### Calculating: Polarity, Coverage, Overlaps and Conflics for the labeling functions

In [49]:
lfs = [
        is_odd,
        is_even, 
        is_two,
        is_known_prime
      ]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df)
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 20/20 [00:00<00:00, 7901.85it/s]


j Polarity  Coverage  Overlaps  Conflicts
is_odd          0      [0]      0.55      0.55       0.05
is_even         1      [0]      0.55      0.55       0.05
is_two          2      [1]      0.05      0.05       0.05
is_known_prime  3      [1]      0.20      0.05       0.05

#### Applying the labeling functions to the dataset, just for illustration of calcualting the above metrics

In [50]:
df["is_odd"] = df.apply(is_odd, axis=1)
df["is_even"] = df.apply(is_even, axis=1)
df["is_two"] = df.apply(is_two, axis=1)
df["is_known_prime"] = df.apply(is_known_prime, axis=1)

In [51]:
df[["Number", "is_odd", "is_even", "is_two", "is_known_prime"]]

Number  is_odd  is_even  is_two  is_known_prime
0        5      -1       -1      -1               1
1       21      -1       -1      -1              -1
2        1      -1       -1      -1              -1
3       29      -1       -1      -1               1
4       32       0        0      -1              -1
5       37      -1       -1      -1              -1
6       10       0        0      -1              -1
7       20       0        0      -1              -1
8       10       0        0      -1              -1
9       26       0        0      -1              -1
10       2       0        0       1               1
11      37      -1       -1      -1              -1
12      34       0        0      -1              -1
13      11      -1       -1      -1               1
14      22       0        0      -1              -1
15      36       0        0      -1              -1
16      12       0        0      -1              -1
17      20       0        0      -1              -1
18      31      -1       -1      -1              -1
19      25      -1       -1      -1              -1

##### Polarity

In [52]:
LFAnalysis(L=L_train, lfs=lfs).lf_polarities()

[[0], [0], [1], [1]]

##### Coverage

In [53]:
print(df[df!= -1].count()/df.shape[0])

Number            1.00
is_odd            0.55
is_even           0.55
is_two            0.05
is_known_prime    0.20
dtype: float64


#### Overlaps

In [54]:
LFAnalysis(L=L_train, lfs=lfs).lf_overlaps()

array([0.55, 0.55, 0.05, 0.05])

#### Conflicts

In [55]:
LFAnalysis(L=L_train, lfs=lfs).lf_conflicts()

array([0.05, 0.05, 0.05, 0.05])

### Calculating: Correct, Incorrect and Empirical Accuracy on the validation set

In [56]:
L_valid = applier.apply(df_val)
LFAnalysis(L_valid, lfs).lf_summary(true_labels)

100%|██████████| 5/5 [00:00<00:00, 4219.62it/s]


j Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  \
is_odd          0      [0]       0.6       0.6        0.2        2          1   
is_even         1      [0]       0.6       0.6        0.2        2          1   
is_two          2      [1]       0.2       0.2        0.2        1          0   
is_known_prime  3      [1]       0.6       0.2        0.2        3          0   

                Emp. Acc.  
is_odd           0.666667  
is_even          0.666667  
is_two           1.000000  
is_known_prime   1.000000

#### Applying the labeling functions to the validationg set, just for illustration of calcualting the above metrics

In [57]:
df_val = pd.DataFrame(validation, columns=['Number'])
df_val["is_odd"] = df_val.apply(is_odd, axis=1)
df_val["is_even"] = df_val.apply(is_even, axis=1)
df_val["is_two"] = df_val.apply(is_two, axis=1)
df_val["is_known_prime"] = df_val.apply(is_known_prime, axis=1)
df_val["ground_truth"] = true_labels
df_val

Number  is_odd  is_even  is_two  is_known_prime  ground_truth
0      22       0        0      -1              -1             0
1      11      -1       -1      -1               1             1
2       7      -1       -1      -1               1             1
3       2       0        0       1               1             1
4      32       0        0      -1              -1             0

### Using Random Voter to determine the label

In [58]:
from snorkel.labeling.model import RandomVoter

In [59]:
random_model = RandomVoter()
preds_train_random = random_model.predict(L=L_train, tie_break_policy='abstain')
preds_valid_random = random_model.predict(L=L_valid)
df["preds_train_random"] = preds_train_random
df[df["Number"] == 2]

Number  is_odd  is_even  is_two  is_known_prime  preds_train_random
10       2       0        0       1               1                   0

Calculating the accuracy of the RandomVoter

In [60]:
metrics = random_model.score(preds_valid_random, true_labels, metrics=['accuracy'])
metrics

{'accuracy': 0.4}

### MajorityClassVoter

In [61]:
from snorkel.labeling.model import MajorityClassVoter

In [62]:
majorityClass_model = MajorityClassVoter()
majorityClass_model.fit(balance=np.array([0.7, 0.3]))
majorityClass_train_random = majorityClass_model.predict(L=L_train)
majorityClass_valid_random = majorityClass_model.predict(L=L_valid)
df["majorityClass_pred"] = majorityClass_train_random
df[df["Number"] == 2]

Number  is_odd  is_even  is_two  is_known_prime  preds_train_random  \
10       2       0        0       1               1                   0   

    majorityClass_pred  
10                   0

In [63]:
metrics = majorityClass_model.score(majorityClass_valid_random, true_labels, metrics=['accuracy'])
metrics

{'accuracy': 0.4}

### Using MajorityLabelVoter to determine the label

In [64]:
majority_model = MajorityLabelVoter()
preds_train_majority = majority_model.predict(L=L_train)
preds_valid_majority = majority_model.predict(L=L_valid)
df["majorityLabel_pred"] = preds_train_majority

In [65]:
df[df["Number"] == 2]

Number  is_odd  is_even  is_two  is_known_prime  preds_train_random  \
10       2       0        0       1               1                   0   

    majorityClass_pred  majorityLabel_pred  
10                   0                  -1

### Using LabelingModel to determine the label

In [66]:
label_model = LabelModel()
label_model.fit(L_train=L_train, n_epochs=200, seed=100)
preds_train_label = label_model.predict(L=L_train)
preds_valid_label = label_model.predict(L=L_valid)
L_valid = applier.apply(df_val)
LFAnalysis(L_valid, lfs).lf_summary()

preds_train_labelingModel = label_model.predict(L=L_train)
preds_valid_labelingModel = label_model.predict(L=L_valid)

100%|██████████| 5/5 [00:00<00:00, 2281.25it/s]


In [67]:
df["preds_labelingModel"] = preds_train_labelingModel

In [68]:
df[["Number", "preds_train_random", "majorityClass_pred", "majorityLabel_pred", "preds_labelingModel"]]

Number  preds_train_random  majorityClass_pred  majorityLabel_pred  \
0        5                   1                   0                   1   
1       21                   0                   0                  -1   
2        1                   1                   0                  -1   
3       29                   0                   0                   1   
4       32                   1                   0                   0   
5       37                   0                   0                  -1   
6       10                   1                   0                   0   
7       20                   0                   0                   0   
8       10                   1                   0                   0   
9       26                   1                   0                   0   
10       2                   0                   0                  -1   
11      37                   0                   0                  -1   
12      34                   0                   0                   0   
13      11                   1                   0                   1   
14      22                   1                   0                   0   
15      36                   0                   0                   0   
16      12                   0                   0                   0   
17      20                   1                   0                   0   
18      31                   1                   0                  -1   
19      25                   1                   0                  -1   

    preds_labelingModel  
0                     1  
1                    -1  
2                    -1  
3                     1  
4                     0  
5                    -1  
6                     0  
7                     0  
8                     0  
9                     0  
10                    0  
11                   -1  
12                    0  
13                    1  
14                    0  
15                    0  
16                    0  
17                    0  
18                   -1  
19                   -1

In [69]:
# 7/20 prime numbers
label_model = LabelModel()
label_model.fit(L_train=L_train, n_epochs=200, class_balance = [0.7, 0.3], seed=100)
preds_train_label = label_model.predict(L=L_train)
preds_valid_label = label_model.predict(L=L_valid)
L_valid = applier.apply(df_val)
LFAnalysis(L_valid, lfs).lf_summary()

preds_train_labelingModel = label_model.predict(L=L_train)
df["preds_labelingModel_wclassBalance"] = preds_train_labelingModel

100%|██████████| 5/5 [00:00<00:00, 3466.94it/s]


In [70]:
df[["Number", "preds_labelingModel", "preds_labelingModel_wclassBalance"]]

Number  preds_labelingModel  preds_labelingModel_wclassBalance
0        5                    1                                  1
1       21                   -1                                  0
2        1                   -1                                  0
3       29                    1                                  1
4       32                    0                                  0
5       37                   -1                                  0
6       10                    0                                  0
7       20                    0                                  0
8       10                    0                                  0
9       26                    0                                  0
10       2                    0                                  0
11      37                   -1                                  0
12      34                    0                                  0
13      11                    1                                  1
14      22                    0                                  0
15      36                    0                                  0
16      12                    0                                  0
17      20                    0                                  0
18      31                   -1                                  0
19      25                   -1                                  0

Looking at the weights of the label_model

In [71]:
np.round(label_model.get_weights(), 2)

array([1.  , 1.  , 0.54, 0.65])

Looking at the conditional probabilities 

In [72]:
np.round(label_model.get_conditional_probs(), 3)

array([[[0.114, 0.98 ],
        [0.876, 0.01 ],
        [0.01 , 0.01 ]],

       [[0.114, 0.98 ],
        [0.876, 0.01 ],
        [0.01 , 0.01 ]],

       [[0.91 , 0.923],
        [0.01 , 0.01 ],
        [0.08 , 0.067]],

       [[0.869, 0.58 ],
        [0.01 , 0.01 ],
        [0.121, 0.41 ]]])